In [1]:
import utils
from utils_data_augmentation import *

## Attempt to reduce data dimensionality with Structural Similarity Index (SSIM)
The entire code is in the file utils.py, it was not reported here because it is refered to an attempt that didn't brought to any result we wanted to keep.
The idea was to remove all the images with an SSIM of 0.95 or higher from the classes with more than 500 elements.
The results were not as expected, so we decided to keep all the images.

In [2]:
utils.create_csv_for_folders('Spettrogrammi/Target/', 8)

In [3]:
utils.create_csv_for_folders('Spettrogrammi/Non-Target/', 8)

In [ ]:
utils.merge_csv_files('Spettrogrammi/Target/dati ssim/SSIM', 'Spettrogrammi/Target/ssim_results.csv')

In [ ]:
utils.merge_csv_files('Spettrogrammi/Non-Target/dati ssim/SSIM', 'Spettrogrammi/Non-Target/ssim_results.csv')

In [ ]:
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/ssim_results.csv')
pd.set_option('display.float_format', '{:.3f}'.format)
df_non_target.describe()

In [ ]:
df_target = pd.read_csv('Spettrogrammi/Target/ssim_results.csv')
pd.set_option('display.float_format', '{:.3f}'.format)
df_target.describe()

In [ ]:
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/df_paths.csv')
df_target = pd.read_csv('Spettrogrammi/Target/df_paths.csv')

df_non_target.to_csv('final_dataset/df_paths_non_target.csv', index=False)
df_target.to_csv('final_dataset/df_paths_target.csv', index=False)

# Split Dataset in Train, validation e Test

### Extracted the numbers of audios for each classes in order to avoid splitting the same audio in the different sets and removed the classes with less than 8 audios.

The results of the operations slightly changes if executed on MacOS or Windows because of how different OS handle the files.

In [2]:
df_target = pd.read_csv('Spettrogrammi/Target/df_paths.csv')
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/df_paths.csv')

In [3]:
utils.estrai_info_da_csv('Spettrogrammi/Target/df_paths.csv', 'Spettrogrammi/Target/df_class_num.csv')
utils.estrai_info_da_csv('Spettrogrammi/Non-Target/df_paths.csv', 'Spettrogrammi/Non-Target/df_class_num.csv')

In [4]:
utils.filter_csv('Spettrogrammi/Target/df_class_num.csv', 'Spettrogrammi/Target/df_paths.csv', 'Spettrogrammi/Target/df_paths_filtered.csv')
utils.filter_csv('Spettrogrammi/Non-Target/df_class_num.csv', 'Spettrogrammi/Non-Target/df_paths.csv', 'Spettrogrammi/Non-Target/df_paths_filtered.csv')

In [5]:
def split_files_into_sets(input_csv_path, train_csv_path, val_csv_path, test_csv_path):
    """
    Split the files in the input CSV into training, validation, and test sets.
    
    :param input_csv_path: the path to the input CSV file
    :param train_csv_path: the path to the output training CSV file
    :param val_csv_path: the path to the output validation CSV file
    :param test_csv_path: the path to the output test CSV file
    :return: None
    """
    np.random.seed(0)

    df = pd.read_csv(input_csv_path)

    df['File'] = df['FilePath'].apply(lambda x: x.split('_resampled')[0])

    df['Set'] = ''

    grouped = df.groupby('Classe')

    for name, group in grouped:
        unique_files = group['File'].unique()

        train_size = int(0.8 * len(unique_files))
        val_size = int(0.1 * len(unique_files))
        test_size = len(unique_files) - train_size - val_size

        file_sets = ['train'] * train_size + ['val'] * val_size + ['test'] * test_size

        np.random.shuffle(file_sets)

        file_to_set = dict(zip(unique_files, file_sets))

        df.loc[group.index, 'Set'] = group['File'].apply(lambda x: file_to_set[x])

    df_train = df[df['Set'] == 'train']
    df_val = df[df['Set'] == 'val']
    df_test = df[df['Set'] == 'test']

    os.makedirs(os.path.dirname(train_csv_path), exist_ok=True)
    os.makedirs(os.path.dirname(val_csv_path), exist_ok=True)
    os.makedirs(os.path.dirname(test_csv_path), exist_ok=True)

    df_train.to_csv(train_csv_path, index=False)
    df_val.to_csv(val_csv_path, index=False)
    df_test.to_csv(test_csv_path, index=False)

In [33]:
split_files_into_sets('Spettrogrammi/Target/df_paths_filtered.csv', 'final_dataset/training/df_paths_target_train.csv', 'final_dataset/validation/df_paths_target_val.csv', 'final_dataset/test/df_paths_target_test.csv')

In [34]:
split_files_into_sets('Spettrogrammi/Non-Target/df_paths_filtered.csv', 'final_dataset/training/df_paths_non_target_train.csv', 'final_dataset/validation/df_paths_non_target_val.csv', 'final_dataset/test/df_paths_non_target_test.csv')

### Distribuzione dei file nel training set

In [2]:
pd.set_option('display.max_rows', None)
df_train_target = pd.read_csv('final_dataset/training/df_paths_target_train.csv')
max_target = np.max(df_train_target['Classe'].value_counts())
max_target

12339

In [3]:
pd.set_option('display.max_rows', None)
df_train_non_target = pd.read_csv('final_dataset/training/df_paths_non_target_train.csv')
max_non_target = np.max(df_train_non_target['Classe'].value_counts())
max_non_target

1822

In [4]:
df_train_target['Classe'].value_counts()

Classe
Tanker           12339
Passengership    11258
Tug              10727
Cargo            10169
Vessel             170
Name: count, dtype: int64

In [5]:
df_train_non_target['Classe'].value_counts()

Classe
Sperm Whale                           1822
Humpback Whale                        1657
Fin, Finback Whale                     818
Soundscape                             239
Fraser_s Dolphin                       224
Long-Finned Pilot Whale                191
Short-Finned (Pacific) Pilot Whale     177
Grampus, Risso_s Dolphin               139
Killer Whale                           133
Spinner Dolphin                        129
Striped Dolphin                        128
Common Dolphin                         111
Bowhead Whale                          107
Melon Headed Whale                      85
Walrus                                  77
Bearded Seal                            75
Atlantic Spotted Dolphin                71
White-sided Dolphin                     70
Harp Seal                               69
Ross Seal                               69
Clymene Dolphin                         68
Pantropical Spotted Dolphin             67
Beluga, White Whale                     65
Nort

## Data Augmentation

In [7]:
process_csv('final_dataset/training/df_paths_target_train.csv', 'final_dataset/training', df_train_target['Classe'].value_counts())

Processing images: 100%|██████████| 44663/44663 [00:36<00:00, 1230.89it/s] 


- bilanciare
- pregare che funzioni

In [40]:
add_rows_to_csv('final_dataset/training/Target', 'final_dataset/training/df_paths_target_train.csv', 'final_dataset/training/new_df_paths_target_train.csv')

In [41]:
add_rows_to_csv('final_dataset/training/Non-Target', 'final_dataset/training/df_paths_non_target_train.csv', 'final_dataset/training/new_df_paths_non_target_train.csv')

In [71]:
merge_csv_files('final_dataset/training/new_df_paths_target_train.csv', 'final_dataset/training/new_df_paths_non_target_train.csv', 'final_dataset/training/df_paths_train.csv')

In [88]:
process_csv_merge('final_dataset/training/df_paths_train.csv', 'final_dataset/training/', df_train_target['Label'].value_counts())

Processing images: 100%|██████████| 121821/121821 [01:21<00:00, 1495.76it/s] 


In [96]:
add_rows_to_csv('final_dataset/training/Non-Target', 'final_dataset/training/df_paths_non_target_train.csv', 'final_dataset/training/new_df_paths_non_target_train.csv')

In [97]:
merge_csv_files('final_dataset/training/new_df_paths_target_train.csv', 'final_dataset/training/new_df_paths_non_target_train.csv', 'final_dataset/training/df_paths_train.csv')

In [98]:
pd.set_option('display.max_rows', None)
df_train_target = pd.read_csv('final_dataset/training/df_paths_train.csv')
# df_train_target
max_target = np.max(df_train_target['Label'].value_counts())
max_target

61695

In [99]:
df_train_target['Label'].value_counts()

Label
Non-Target    61695
Target        61695
Name: count, dtype: int64

In [4]:
# Unione file csv per il validation set
merge_csv_files('final_dataset/validation/df_paths_target_val.csv', 'final_dataset/validation/df_paths_non_target_val.csv', 'final_dataset/validation/df_paths_val.csv')

In [6]:
count_files_in_folder('/Users/daniela/Library/CloudStorage/OneDrive-UniversitàdiSalerno/I Anno/Secondo Semestre/Fondamenti di Visione Artificiale e Biometria/Progetto/biometria/final_dataset/Non-Target')

54679

In [4]:
merge_csv_files('final_dataset/training/df_paths_target_train.csv', 'final_dataset/training/df_paths_non_target_train.csv', 'final_dataset/training/df_paths_train_clean.csv')

In [3]:
merge_csv_files('final_dataset/test/df_paths_target_test.csv', 'final_dataset/test/df_paths_non_target_test.csv', 'final_dataset/test/df_paths_test.csv')